In [1]:
import pandas as pd
import numpy as np
import decimal as D

In [2]:
COIN = 100 * 1000 * 1000

In [3]:
pd.set_option('display.float_format', lambda x: '%.9f' % x)

In [4]:
df = pd.read_csv('btc_audit_data_v2.csv', low_memory=False, 
              converters={
                  'total_amount': D.Decimal, 
                  'total_unspendable_amount': D.Decimal,
                  'block_info_unspendable_amount': D.Decimal,
                  'block_info_total_prevout_spent_amount': D.Decimal,
                  'block_info_total_new_outputs_ex_coinbase_amount': D.Decimal,
                  'block_info_coinbase_amount': D.Decimal,
                         }
                )  

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644829 entries, 0 to 644828
Data columns (total 11 columns):
 #   Column                                           Non-Null Count   Dtype 
---  ------                                           --------------   ----- 
 0   height                                           644829 non-null  int64 
 1   bestblock                                        644829 non-null  object
 2   txouts                                           644829 non-null  int64 
 3   bogosize                                         644829 non-null  int64 
 4   disk_size                                        644829 non-null  int64 
 5   total_amount                                     644829 non-null  object
 6   total_unspendable_amount                         644829 non-null  object
 7   block_info_unspendable_amount                    644829 non-null  object
 8   block_info_total_prevout_spent_amount            644829 non-null  object
 9   block_info_total_new_outpu

In [6]:
# Are there any missing blocks?
seq = pd.RangeIndex(df.height.min(), df.height.max())
len(seq[~seq.isin(df.height)].values)

0

In [7]:
# Did any unspendables not get added to the total?
df['total_expected_unspendable_amount'] = df.block_info_unspendable_amount.cumsum()
df['reconcile_unspendables'] = df['total_expected_unspendable_amount'] - df.total_unspendable_amount 
len(df[df.reconcile_unspendables != 0])

0

In [27]:
df['imputed_tx_fees'] = df.block_info_total_prevout_spent_amount - df.block_info_total_new_outputs_ex_coinbase_amount

In [33]:
df['expected_issuance'] = (df.height.apply(lambda x: D.Decimal(50 * COIN >> int(x/210000)))) / D.Decimal(COIN)

In [34]:
df['total_expected_issuance'] = df.expected_issuance.cumsum()

In [35]:
df[df.reconciliation != 0]

,height,bestblock,txouts,bogosize,disk_size,total_amount,total_unspendable_amount,block_info_unspendable_amount,block_info_total_prevout_spent_amount,block_info_total_new_outputs_ex_coinbase_amount,block_info_coinbase_amount,total_expected_unspendable_amount,reconcile_unspendables,expected_issuance,total_expected_issuance,expected_spendable,reconciliation,imputed_tx_fees


In [37]:
df.sort_values(['imputed_tx_fees'], ascending=[1])

,height,bestblock,txouts,bogosize,disk_size,total_amount,total_unspendable_amount,block_info_unspendable_amount,block_info_total_prevout_spent_amount,block_info_total_new_outputs_ex_coinbase_amount,block_info_coinbase_amount,total_expected_unspendable_amount,reconcile_unspendables,expected_issuance,total_expected_issuance,expected_spendable,reconciliation,imputed_tx_fees
166732,166732,0000000000000af7c56d09abdbce2a36ba9aebbd8559b3...,1331110,101599468,4762892918,8336490.57589413,159.42410587,0E-8,756.69056678,806.69056678,0E-8,159.42410587,0E-8,50,8336650,8336490.57589413,0E-8,-50.00000000
168984,168984,000000000000079e7d4488da81e7793d8a0fadad748d90...,1374080,104828237,4762515118,8449090.41075376,159.58924624,0E-8,409.92880640,459.92880640,0E-8,159.58924624,0E-8,50,8449250,8449090.41075376,0E-8,-50.00000000
166984,166984,000000000000041bd7cb486229b347f9a25a875ebb109e...,1335399,101917569,4762892918,8349090.56808977,159.43191023,0E-8,4346.85021109,4396.85021109,0E-8,159.43191023,0E-8,50,8349250,8349090.56808977,0E-8,-50.00000000
166533,166533,00000000000001ee89122dcb6ebe8ae920a9bb36ad17f5...,1326002,101216924,4762892918,8326540.58989413,159.41010587,0E-8,3238.33994858,3288.33994858,0E-8,159.41010587,0E-8,50,8326700,8326540.58989413,0E-8,-50.00000000
168978,168978,0000000000000a97116e69aa41fd2d4b4ae529fb330000...,1373936,104817364,4762515118,8448790.41075376,159.58924624,0E-8,85.06287675,135.06287675,0E-8,159.58924624,0E-8,50,8448950,8448790.41075376,0E-8,-50.00000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201731,201731,0000000000000412cb93f5cceceebdd6bec88d5db691a1...,2414953,182885400,4760517484,10086439.98261183,160.01738817,0E-8,25336.11222473,25231.42923262,154.68299211,160.01738817,0E-8,50,10086600,10086439.98261183,0E-8,104.68299211
215936,215936,00000000000002327a29e7fdf539e773e960e631622c46...,3335831,251952569,4760517484,10648264.89651183,160.10348817,0E-8,10842.94486984,10731.45411984,136.49075000,160.10348817,0E-8,25,10648425,10648264.89651183,0E-8,111.49075000
157235,157235,00000000000009736897988613310282732f87d22a2253...,1232993,94255308,4763136101,7861649.98999999,150.01000001,0E-8,5588.51853952,5416.67204429,221.84649523,150.01000001,0E-8,50,7861800,7861649.98999999,0E-8,171.84649523
254642,254642,0000000000000020c334379c4642537b5ab82f4a01e092...,6884933,521012070,4760517484,11615914.79581183,160.20418817,0E-8,5941.94694186,5741.72093969,225.22600217,160.20418817,0E-8,25,11616075,11615914.79581183,0E-8,200.22600217
